In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import math

In [3]:
df_track = pd.read_csv("device_track_full.csv")
df_elev = pd.read_csv("track_with_elevation.csv")

In [4]:
df_track.head()

,track_id,device_id,timestamp,longitude,latitude,battery_level,emergency_status,condition,off_track,segment_id
0,T-WAT_1-1762845227.0,WAT_1,2025-11-11T07:13:47,109.959301,-7.224694,65.96,False,badai petir,False,771.0
1,T-WAT_1-1762845287.0,WAT_1,2025-11-11T07:14:47,109.958848,-7.224514,65.91,False,badai petir,False,778.0
2,T-WAT_1-1762845347.0,WAT_1,2025-11-11T07:15:47,109.958525,-7.224461,65.90,False,badai petir,False,780.0
3,T-WAT_1-1762845407.0,WAT_1,2025-11-11T07:16:47,109.958535,-7.224219,65.85,False,badai petir,False,783.0
4,T-WAT_1-1762845467.0,WAT_1,2025-11-11T07:17:47,109.958295,-7.223958,65.81,False,badai petir,False,788.0


In [5]:
df_track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711540 entries, 0 to 711539
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          711540 non-null  object 
 1   device_id         711540 non-null  object 
 2   timestamp         711540 non-null  object 
 3   longitude         711540 non-null  float64
 4   latitude          711540 non-null  float64
 5   battery_level     711540 non-null  float64
 6   emergency_status  711540 non-null  bool   
 7   condition         711540 non-null  object 
 8   off_track         711540 non-null  bool   
 9   segment_id        711540 non-null  float64
dtypes: bool(2), float64(4), object(4)
memory usage: 44.8+ MB


In [6]:
df_track["timestamp"] = pd.to_datetime(df_track["timestamp"])

In [ ]:
df_track_1 = df_track[df_track['timestamp'].dt.date == pd.to_datetime("2025-11-11").date()].copy()
df_track_1.head()

In [ ]:
df_track_2 = df_track[df_track['timestamp'].dt.date == pd.to_datetime("2025-11-12").date()].copy()
df_track_2.head()

In [ ]:
df_track_3 = df_track[df_track['timestamp'].dt.date == pd.to_datetime("2025-11-13").date()].copy()
df_track_3.head()

In [ ]:
df_elev.head()

In [ ]:
# pastikan kolom terstandarisasi namanya
df_elev.rename(columns={ 
    "elevation": "ele"
}, inplace=True)

In [ ]:
df_elev.info()

In [ ]:
# ==========================================================
# 2. HITUNG FITUR SEGMENT-LEVEL
# ==========================================================
# Setiap segment = titik i → i+1 dalam track_with_elevation

segments = []

for i in range(len(df_elev) - 1):
    lat1, lon1, ele1 = df_elev.loc[i, ["lat", "lon", "ele"]]
    lat2, lon2, ele2 = df_elev.loc[i+1, ["lat", "lon", "ele"]]

    # -----------------------------
    # a) segment length (meters)
    # -----------------------------
    R = 6371000
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = (math.sin(dphi/2)**2 +
         math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2)
    dist = 2 * R * math.atan2(np.sqrt(a), np.sqrt(1 - a))

    # -----------------------------
    # b) slope (elevation gain / distance)
    # -----------------------------
    if dist == 0:
        slope = 0
    else:
        slope = (ele2 - ele1) / dist  # meters per meter → unitless

    # -----------------------------
    # c) curvature (perubahan heading)
    # -----------------------------
    # heading i → i+1
    heading1 = math.atan2(
        math.radians(lon2 - lon1),
        math.radians(lat2 - lat1)
    )

    # heading i+1 → i+2 (kalau ada)
    if i < len(df_elev) - 2:
        lat3, lon3 = df_elev.loc[i+2, ["lat", "lon"]]
        heading2 = math.atan2(
            math.radians(lon3 - lon2),
            math.radians(lat3 - lat2)
        )
        curvature = abs(heading2 - heading1)
    else:
        curvature = 0  # segment terakhir

    # simpan
    segments.append({
        "segment_id": i,
        "lat": lat1,
        "lon": lon1,
        "length_m": dist,
        "slope": slope,
        "curvature": curvature
    })

df_seg = pd.DataFrame(segments)

In [ ]:
df_seg.head()

In [ ]:
# ==========================================================
# 3. GABUNG DENGAN device_track (density, stuck, offtrack)
# ==========================================================

# hitung jumlah device per segmen berdasarkan kedekatan koordinat
def closest_segment(lat, lon):
    d = (df_seg["lat"] - lat)**2 + (df_seg["lon"] - lon)**2
    return d.idxmin()

df_track_1["segment_id"] = df_track_1.apply(
    lambda row: closest_segment(row["latitude"], row["longitude"]), axis=1
)

In [ ]:
df_track_1[:10]

In [ ]:
# hitung fitur tambahan
seg_stats = df_track_1.groupby("segment_id").agg({
    "device_id": "count",
    "off_track": "mean"
}).rename(columns={
    "device_id": "density",
    "off_track": "offtrack_rate"
}).reset_index()

df_seg = df_seg.merge(seg_stats, on="segment_id", how="left")

df_seg.fillna({
    "density": 0,
    "offtrack_rate": 0
}, inplace=True)

In [ ]:
features = df_seg[["length_m", "slope", "curvature", "density", "offtrack_rate"]]

# Standarisasi fitur
scaler = StandardScaler()
X = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto")
kmeans.fit(X)

# Label cluster
difficulty_map = {
     0: "Noise / Outlier",
     1: "Mudah",
     2: "Sedang",
     3: "Sulit",
     4: "Sangat Sulit"
}

df_seg["difficulty"] = df_seg["cluster"].map(difficulty_map).fillna("Sulit")

In [ ]:
df_seg[:10]

In [ ]:
diff_count = df_seg.groupby("cluster").agg({
    "segment_id": "count"
})
print(diff_count)

In [ ]:
import matplotlib.pyplot as plt

difficulty = df_seg['cluster']
norm = (difficulty - difficulty.min()) / (difficulty.max() - difficulty.min() + 1e-9)

plt.figure(figsize=(10, 8))
sc = plt.scatter(
    df_seg["lon"],
    df_seg["lat"],
    c=norm,
    cmap="plasma",     # bisa diganti viridis, inferno, turbo, magma, dll
    s=25,
    edgecolor="none"
)

plt.colorbar(sc, label="Difficulty Level (Normalized)")
plt.title("Visualisasi Kesulitan Segmen Jalur Gunung (DBSCAN)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True, alpha=0.3)

plt.show()

In [ ]:
outlier = df_seg.loc[df_seg['cluster'] == -1]

plt.figure(figsize=(8, 6))
sc = plt.scatter(
    outlier["lon"],
    outlier["lat"],     
    edgecolor="none"
)

plt.title("Noise")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True, alpha=0.3)

plt.show()

In [ ]:
outlier

In [ ]:
from sklearn import metrics

labels = db.labels_
sc = metrics.silhouette_score(X, labels)
print("Silhouette Coefficient:%0.2f" % sc)

In [ ]:
# ==========================================================
# 5. SAVE OUTPUT
# ==========================================================

# df_seg.to_csv("segment_difficulty_dbscan.csv", index=False)
# print("Saved: segment_difficulty_dbscan.csv")